# Analyses in R

## Information
Project: **The price of being late: short- and long-term consequences of a delayed migration timing**  
Author: Iris Bontekoe  
Program: R 4.1.2  
Description: This script contains all analyses carried out to obtain the results in the manuscript.

### Preparations

In [ ]:
# Change work directory
setwd("[...]")

# Set standard time zone to UTC
Sys.setenv(TZ="UTC")

# Load packages
library(lubridate)
#library(lme4)
library(multcomp)
library(dplyr)
library(lmerTest)
library(pbkrtest)
library(rstatix)

## Analyses

### # storks in segment

In [ ]:
# Load data
data<-read.csv("DATA/All_DaysInSegment.csv",header=T,sep=",",na.strings=c("","NA"))
data$Year<-year(data$Timestamp_first)

In [ ]:
# Affenberg
print(paste0("Affenberg: ",length(unique(data[data$Aviary=="Affenberg",]$Individual))))

# Care center
print(paste0("Care center: ",length(unique(data[data$Aviary=="CareCenter",]$Individual))))

# CASCB
print(paste0("CASCB: ",length(unique(data[data$Aviary=="CASCB",]$Individual))))

### # migrants and non-migrants

In [ ]:
# Load data
data<-read.csv("DATA/All_SegTimMinLat.csv",header=T,sep=",",na.strings=c("","NA"))

# Remove all data that does not have a MinLatDate1 (i.e. remove individuals that did not survive)
data<-data[!(is.na(data$MinLatDate1)),]

# Determine which individuals migrated
data$Mig<-data$Distance1>200

# Add year to the data
data$Year<-year(data$FirstDate1)

In [ ]:
# Get the total number of data points
nrow(data)

In [ ]:
# Calculate the number of migrating and non-migrating individuals
data2<-matrix(nrow=3,ncol=2)
rownames(data2)<-c("Affenberg","CareCenter","CASCB")
colnames(data2)<-c("Migrant","Non-migrant")

data2["Affenberg","Migrant"]<-nrow(data[data$Aviary=="Affenberg"&data$Mig,])
data2["Affenberg","Non-migrant"]<-nrow(data[data$Aviary=="Affenberg"&!data$Mig,])

data2["CareCenter","Migrant"]<-nrow(data[data$Aviary=="CareCenter"&data$Mig,])
data2["CareCenter","Non-migrant"]<-nrow(data[data$Aviary=="CareCenter"&!data$Mig,])

data2["CASCB","Migrant"]<-nrow(data[data$Aviary=="CASCB"&data$Mig,])
data2["CASCB","Non-migrant"]<-nrow(data[data$Aviary=="CASCB"&!data$Mig,])

print(data2)

In [ ]:
# Calculate the number of migrants and the total number for both years of Affenberg
print(paste0(
    "Migrants 2019: ",
    nrow(data[data$Aviary=="Affenberg"&data$Mig&data$Year==2019,])
))

print(paste0(
    "Total 2019: ",
    nrow(data[data$Aviary=="Affenberg"&data$Year==2019,])
))

print(paste0(
    "Migrants 2020: ",
    nrow(data[data$Aviary=="Affenberg"&data$Mig&data$Year==2020,])
))

print(paste0(
    "Total 2020: ",
    nrow(data[data$Aviary=="Affenberg"&data$Year==2020,])
))

In [ ]:
# Test the difference between groups
cat("\n------------------------- Test for differences -------------------------\n")
fisher.test(data2)
cat("\n---------------------------- Post-hoc test -----------------------------\n")
pairwise_fisher_test(data2,p.adjust.method="fdr")
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

### Migration distance

In [ ]:
# Load data
data<-read.csv("DATA/All_SegTimMinLat.csv",header=T,sep=",",na.strings=c("","NA"))

# Remove all data that does not have a Distance1
data<-data[!(is.na(data$Distance1)),]

In [ ]:
# Extract the number of individuals
# Affenberg
print(paste0("Affenberg: ",length(unique(data[data$Aviary=="Affenberg"&data$Distance1>200,]$Individual))))

# Care center
print(paste0("Care center: ",length(unique(data[data$Aviary=="CareCenter"&data$Distance1>200,]$Individual))))

# CASCB
print(paste0("CASCB: ",length(unique(data[data$Aviary=="CASCB"&data$Distance1>200,]$Individual))))

In [ ]:
# Calculate the mean and standard deviation
MeanSD<-aggregate(Distance1~Aviary,data[data$Distance1>200,],function(x) c(mean = mean(x), sd = sd(x)))
MeanSD$Mean<-MeanSD$DistFinal[,1]
MeanSD$SD<-MeanSD$DistFinal[,2]
MeanSD$DistFinal<-NULL

print(MeanSD)

In [ ]:
# Get the total number of data points
nrow(data[data$Distance1>200,])

In [ ]:
# Test model assumptions
# Test homogeneity of variance and normality
Model_groups<-lm(Distance1~Aviary,data=data[data$Distance1>200,])
par(mfrow=c(2,2))

plot(Model_groups)

In [ ]:
# This doesnt't look too good. Try transforming the data
data$Distance1_log<-log(data$Distance1)
data$Distance1_sqrt<-sqrt(data$Distance1)

In [ ]:
# Test homogeneity of variance and normality
Model_groups<-lm(Distance1_log~Aviary,data=data[data$Distance1>200,])
par(mfrow=c(2,2))

plot(Model_groups)

In [ ]:
# Test homogeneity of variance and normality
Model_groups<-lm(Distance1_sqrt~Aviary,data=data[data$Distance1>200,])
par(mfrow=c(2,2))

plot(Model_groups)

In [ ]:
# That doesn't help

In [ ]:
# Test differences between groups
data$Aviary2<-as.factor(data$Aviary)
Model_groups<-lm(Distance1~Aviary2,data=data[data$Distance1>200,])

cat("\n--------------------------- Summary of model ---------------------------\n")
summary(Model_groups)
cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups)
cat("\n------------------------- Post-hoc Tukey test --------------------------\n")
summary(glht(Model_groups,linfct=mcp(Aviary2="Tukey")))
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

### % continuing to sub-Saharan Africa

In [ ]:
# Load data
data<-read.csv("DATA/All_SegTimMinLat.csv",header=T,sep=",",na.strings=c("","NA"))

# Remove all data that does not have a Distance1 (i.e. remove individuals that did not finish migration)
data<-data[!(is.na(data$Distance1)),]

# Determine which individuals migrated to sub-Saharan Africa
data$SubSah<-data$Distance1>3000

In [ ]:
# Get the total number of data points
nrow(data)

In [ ]:
# Extract the number of individuals
# Affenberg
print(paste0("Affenberg: ",length(unique(data[data$Aviary=="Affenberg",]$Individual))))

# Care center
print(paste0("Care center: ",length(unique(data[data$Aviary=="CareCenter",]$Individual))))

# CASCB
print(paste0("CASCB: ",length(unique(data[data$Aviary=="CASCB",]$Individual))))

In [ ]:
# Calculate the percentage of individuals that migrate to sub-Saharan Africa (threshold: 3000 km)
print(paste0(
    "Affenberg: ",
    round(nrow(data[data$Aviary=="Affenberg"&data$SubSah,])/nrow(data[data$Aviary=="Affenberg",])*100,digits=1),
    "%"
))

print(paste0(
    "Care center: ",
    round(nrow(data[data$Aviary=="CareCenter"&data$SubSah,])/nrow(data[data$Aviary=="CareCenter",])*100,digits=1),
    "%"
))

print(paste0(
    "CASCB: ",
    round(nrow(data[data$Aviary=="CASCB"&data$SubSah,])/nrow(data[data$Aviary=="CASCB",])*100,digits=1),
    "%"
))

In [ ]:
# Calculate the number of migrating and non-migrating individuals
data2<-matrix(nrow=3,ncol=3)
rownames(data2)<-c("Affenberg","CareCenter","CASCB")
colnames(data2)<-c("SubSah","NSah","Total")

data2["Affenberg","SubSah"]<-nrow(data[data$Aviary=="Affenberg"&data$SubSah,])
data2["Affenberg","NSah"]<-nrow(data[data$Aviary=="Affenberg"&!data$SubSah,])
data2["Affenberg","Total"]<-nrow(data[data$Aviary=="Affenberg",])

data2["CareCenter","SubSah"]<-nrow(data[data$Aviary=="CareCenter"&data$SubSah,])
data2["CareCenter","NSah"]<-nrow(data[data$Aviary=="CareCenter"&!data$SubSah,])
data2["CareCenter","Total"]<-nrow(data[data$Aviary=="CareCenter",])

data2["CASCB","SubSah"]<-nrow(data[data$Aviary=="CASCB"&data$SubSah,])
data2["CASCB","NSah"]<-nrow(data[data$Aviary=="CASCB"&!data$SubSah,])
data2["CASCB","Total"]<-nrow(data[data$Aviary=="CASCB",])

print(data2)

In [ ]:
# Calculate the number of migrating and non-migrating individuals
data2<-matrix(nrow=3,ncol=2)
rownames(data2)<-c("Affenberg","CareCenter","CASCB")
colnames(data2)<-c("SubSah","NSah")

data2["Affenberg","SubSah"]<-nrow(data[data$Aviary=="Affenberg"&data$SubSah,])
data2["Affenberg","NSah"]<-nrow(data[data$Aviary=="Affenberg"&!data$SubSah,])

data2["CareCenter","SubSah"]<-nrow(data[data$Aviary=="CareCenter"&data$SubSah,])
data2["CareCenter","NSah"]<-nrow(data[data$Aviary=="CareCenter"&!data$SubSah,])

data2["CASCB","SubSah"]<-nrow(data[data$Aviary=="CASCB"&data$SubSah,])
data2["CASCB","NSah"]<-nrow(data[data$Aviary=="CASCB"&!data$SubSah,])

In [ ]:
# Test the difference between groups
cat("\n------------------------- Test for differences -------------------------\n")
fisher.test(data2)
cat("\n---------------------------- Post-hoc test -----------------------------\n")
pairwise_fisher_test(data2,p.adjust.method="fdr")
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

### Differences in wintering latitude and timing in Year 1 + Year 2

In [ ]:
# Load data
data<-read.csv("DATA/All_SegTimMinLat.csv",header=T,sep=",",na.strings=c("","NA"))
data$DOY1<-yday(as.POSIXct(data$FirstSegm1))
data$DOY2<-yday(as.POSIXct(data$FirstSegm2))

In [ ]:
# Get the number of individuals

# Affenberg
print(paste0("Affenberg year 1: ",length(unique(data[data$Aviary=="Affenberg"&!(is.na(data$MinLat1)),]$Individual))))

# Care center
print(paste0("Care center year 1: ",length(unique(data[data$Aviary=="CareCenter"&!(is.na(data$MinLat1)),]$Individual))))

# CASCB
print(paste0("CASCB year 1: ",length(unique(data[data$Aviary=="CASCB"&!(is.na(data$MinLat1)),]$Individual))))


# Affenberg
print(paste0("Affenberg year 2: ",length(unique(data[data$Aviary=="Affenberg"&!(is.na(data$MinLat2)),]$Individual))))

# Care center
print(paste0("Care center year 2: ",length(unique(data[data$Aviary=="CareCenter"&!(is.na(data$MinLat2)),]$Individual))))

# CASCB
print(paste0("CASCB year 2: ",length(unique(data[data$Aviary=="CASCB"&!(is.na(data$MinLat2)),]$Individual))))

In [ ]:
#----------#
#- Year 1 -#
#----------#

# Check model assumptions
# Test homogeneity of variance and normality
Model_groups<-lm(MinLat1~Aviary,data=data)
options(repr.plot.width = 10, repr.plot.height = 10);par(mfrow=c(2,2))

plot(Model_groups)

In [ ]:
# Test differences between groups
data$Aviary2<-as.factor(data$Aviary)
Model_groups<-lm(MinLat1~Aviary2,data=data)

cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups)
cat("\n------------------------- Post-hoc Tukey test --------------------------\n")
summary(glht(Model_groups,linfct=mcp(Aviary2="Tukey")))
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

In [ ]:
# Get the total number of data points
nrow(data[!(is.na(data$MinLat1)),])

In [ ]:
#----------#
#- Year 2 -#
#----------#

# Check model assumptions
# Test homogeneity of variance and normality
Model_groups<-lm(MinLat2~Aviary,data=data)
options(repr.plot.width = 10, repr.plot.height = 10);par(mfrow=c(2,2))

plot(Model_groups)

In [ ]:
# Test differences between groups
data$Aviary2<-as.factor(data$Aviary)
Model_groups<-lm(MinLat2~Aviary2,data=data)

cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups)
cat("\n------------------------- Post-hoc Tukey test --------------------------\n")
summary(glht(Model_groups,linfct=mcp(Aviary2="Tukey")))
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

In [ ]:
# Get the total number of data points
nrow(data[!(is.na(data$MinLat2)),])

In [ ]:
#----------#
#- Year 1 -#
#----------#

# Check model assumptions
# Test homogeneity of variance and normality
Model_groups<-lm(DOY1~Aviary,data=data)
options(repr.plot.width = 10, repr.plot.height = 10);par(mfrow=c(2,2))

plot(Model_groups)

In [ ]:
# Test differences between groups
data$Aviary2<-as.factor(data$Aviary)
Model_groups<-lm(DOY1~Aviary2,data=data)

cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups)
cat("\n------------------------- Post-hoc Tukey test --------------------------\n")
summary(glht(Model_groups,linfct=mcp(Aviary2="Tukey")))
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

In [ ]:
# Get the total number of data points
nrow(data[!(is.na(data$DOY1)),])

In [ ]:
#----------#
#- Year 2 -#
#----------#

# Check model assumptions
# Test homogeneity of variance and normality
Model_groups<-lm(DOY2~Aviary,data=data)
options(repr.plot.width = 10, repr.plot.height = 10);par(mfrow=c(2,2))

plot(Model_groups)

In [ ]:
# Test differences between groups
data$Aviary2<-as.factor(data$Aviary)
Model_groups<-lm(DOY2~Aviary2,data=data)

cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups)
cat("\n------------------------- Post-hoc Tukey test --------------------------\n")
summary(glht(Model_groups,linfct=mcp(Aviary2="Tukey")))
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

In [ ]:
# Get the total number of data points
nrow(data[!(is.na(data$DOY2)),])

### Survival

In [ ]:
# Load data
data<-read.table("DATA/All_Survival.csv",header=T,sep=",",na.strings=c("","NA"))
data$timestamp<-as.POSIXct(data$timestamp)
data$Death<-as.POSIXct(data$Death)

# Allign the data
data$timestamp2<-data$timestamp
data[data$StartYear==2020,]$timestamp2<-data[data$StartYear==2020,]$timestamp-as.difftime(366,units="days")

data$Death2<-data$Death
data[data$StartYear==2020,]$Death2<-data[data$StartYear==2020,]$Death-as.difftime(366,units="days")

# Calculate survival over time
# Affenberg
data_Aff<-data.frame(Day=seq(as.POSIXct("2019-07-01"),as.POSIXct("2020-06-30"),by="day"),Aviary="Affenberg",NumTot=length(unique(data[data$Aviary=="Affenberg",]$Individual)),NumSurv=NA,Survival=NA)
data_h1<-aggregate(Death2~Individual,data[data$Aviary=="Affenberg",],first)
data_h1$Day<-floor_date(data_h1$Death2,unit ="days")
data_h1<-aggregate(Individual~Day,data_h1,length)
data_Aff<-merge(data_Aff,data_h1,by="Day",all.x=T)
data_Aff[is.na(data_Aff$Individual),]$Individual<-0
data_Aff$Mortality<-cumsum(data_Aff$Individual)
data_Aff$NumSurv<-data_Aff$NumTot-data_Aff$Mortality
data_Aff$Survival<-data_Aff$NumSurv/data_Aff$NumTot

# Care center
data_CC<-data.frame(Day=seq(as.POSIXct("2019-07-01"),as.POSIXct("2020-06-30"),by="day"),Aviary="CareCenter",NumTot=length(unique(data[data$Aviary=="CareCenter",]$Individual)),NumSurv=NA,Survival=NA)
data_h1<-aggregate(Death2~Individual,data[data$Aviary=="CareCenter",],first)
data_h1$Day<-floor_date(data_h1$Death2,unit ="days")
data_h1<-aggregate(Individual~Day,data_h1,length)
data_CC<-merge(data_CC,data_h1,by="Day",all.x=T)
data_CC[is.na(data_CC$Individual),]$Individual<-0
data_CC$Mortality<-cumsum(data_CC$Individual)
data_CC$NumSurv<-data_CC$NumTot-data_CC$Mortality
data_CC$Survival<-data_CC$NumSurv/data_CC$NumTot

# CASCB
data_CASCB<-data.frame(Day=seq(as.POSIXct("2019-07-01"),as.POSIXct("2020-06-30"),by="day"),Aviary="CASCB",NumTot=length(unique(data[data$Aviary=="CASCB",]$Individual)),NumSurv=NA,Survival=NA)
data_h1<-aggregate(Death2~Individual,data[data$Aviary=="CASCB",],first)
data_h1$Day<-floor_date(data_h1$Death2,unit ="days")
data_h1<-aggregate(Individual~Day,data_h1,length)
data_CASCB<-merge(data_CASCB,data_h1,by="Day",all.x=T)
data_CASCB[is.na(data_CASCB$Individual),]$Individual<-0
data_CASCB$Mortality<-cumsum(data_CASCB$Individual)
data_CASCB$NumSurv<-data_CASCB$NumTot-data_CASCB$Mortality
data_CASCB$Survival<-data_CASCB$NumSurv/data_CASCB$NumTot

# Merge the data for the three studies
data_survival<-rbind(data_Aff,data_CC,data_CASCB)

# Set the surival to 0 on the days before the first release
data_survival[data_survival$Aviary=="Affenberg"&data_survival$Day<as.POSIXct("2019-09-13"),]$Survival<-NA
data_survival[data_survival$Aviary=="CareCenter"&data_survival$Day<as.POSIXct("2019-08-11"),]$Survival<-NA

# Extract the number of surviving storks
aggregate(cbind(NumSurv,NumTot)~Aviary,data_survival,tail,1)

### # days in segment

In [ ]:
# Load data
data<-read.csv("DATA/All_DaysInSegment.csv",header=T,sep=",",na.strings=c("","NA"))

In [ ]:
# Get the total number of data points
nrow(data)

In [ ]:
# Get the number of individuals per group
aggregate(tag.local.identifier~Aviary,data,length)

In [ ]:
# Calculate the maximum, minimum and median
print(paste0("CASCB - max: ",round(max(data[data$Aviary=="CASCB",]$Days),digits=1)))
print(paste0("CASCB - min: ",round(min(data[data$Aviary=="CASCB",]$Days),digits=1)))
print(paste0("CASCB - median: ",round(median(data[data$Aviary=="CASCB",]$Days),digits=1)))

print(paste0("Care center - max: ",round(max(data[data$Aviary=="CareCenter",]$Days),digits=1)))
print(paste0("Care center - min: ",round(min(data[data$Aviary=="CareCenter",]$Days),digits=1)))
print(paste0("Care center - median: ",round(median(data[data$Aviary=="CareCenter",]$Days),digits=1)))

print(paste0("Affenberg - max: ",round(max(data[data$Aviary=="Affenberg",]$Days),digits=1)))
print(paste0("Affenberg - min: ",round(min(data[data$Aviary=="Affenberg",]$Days),digits=1)))
print(paste0("Affenberg - median: ",round(median(data[data$Aviary=="Affenberg",]$Days),digits=1)))

In [ ]:
# Test differences between groups
kruskal.test(Days~Aviary,data = data)

In [ ]:
# Test pairwise differences
pairwise.wilcox.test(data$Days,data$Aviary,p.adjust.method = "BH")

### # stopover days in segment

In [ ]:
# Load data
data<-read.csv("DATA/All_DailyDistance.csv",header=T,sep=",",na.strings=c("","NA"))

# Convert timestamps
data$Day2<-as.POSIXct(data$Day)

# Calculate the number of stopover days
df <- data.frame(matrix(ncol = 4, nrow = 0))
colnames(df)=c("Aviary","Individual","tag.local.identifier","NoStopoverDays")

count<-0
for(i in unique(data$Individual)){
    count<-count+1
    data2<-data[data$Individual==i,]
    
    df[count,"Aviary"]<-unique(data2$Aviary)
    df[count,"Individual"]<-unique(data2$Individual)
    df[count,"tag.local.identifier"]<-unique(data2$tag.local.identifier)
    
    MinDay<-min(data2$Day2)
    MaxDay<-max(data2$Day2)
    SeQuence<-seq(MinDay,MaxDay,by="days")
    NoStopoverDays<-length(SeQuence[!(SeQuence %in% data2$Day2)])
    df[count,"NoStopoverDays"]<-NoStopoverDays
}

In [ ]:
# Get the total number of data points
nrow(df)

In [ ]:
# Test differences between groups
kruskal.test(NoStopoverDays~Aviary,data = df)

In [ ]:
# Test pairwise differences
pairwise.wilcox.test(df$NoStopoverDays,df$Aviary,p.adjust.method = "BH")

### Route straightness

In [ ]:
# Load data
data<-read.csv("DATA/All_EfficiencySegment.csv",header=T,sep=",",na.strings=c("","NA"))

In [ ]:
# Get the total number of data points
nrow(data)

In [ ]:
# Check model assumptions
# Test homogeneity of variance and normality
Model_groups<-lm(Straightness_Segm~Aviary,data=data)
par(mfrow=c(2,2))

plot(Model_groups)

In [ ]:
# That looks fine

In [ ]:
# Test differences between groups
data$Aviary2<-as.factor(data$Aviary)
Model_groups<-lm(Straightness_Segm~Aviary2,data=data)

cat("\n--------------------------- Summary of model ---------------------------\n")
summary(Model_groups)
cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups)
cat("\n------------------------- Post-hoc Tukey test --------------------------\n")
summary(glht(Model_groups,linfct=mcp(Aviary2="Tukey")))
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

### Day length

In [ ]:
# Load data
data<-read.csv("DATA/All_DailyFlightTime.csv",header=T,sep=",",na.strings=c("","NA"))

In [ ]:
# Get the total number of data points
nrow(data)

In [ ]:
# Check model assumptions
# Test homogeneity of variance and normality
Model_groups<-lmer(DayLength~Aviary+(1|Individual),data=data)
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# This doesn't look too bad

In [ ]:
# Test for differences between the groups
Model_groups<-lmer(DayLength~Aviary+(1|Individual),data=data)

cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups,ddf="Kenward-Roger")
cat("\n------------------------- Post-hoc Tukey test --------------------------\n")
summary(glht(Model_groups,linfct=mcp(Aviary="Tukey")))
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

### Daily flight time

In [ ]:
# Load data
data<-read.csv("DATA/All_DailyFlightTime.csv",header=T,sep=",",na.strings=c("","NA"))

In [ ]:
# Get the total number of data points
nrow(data)

In [ ]:
# Check model assumptions
# Test homogeneity of variance and normality
Model_groups<-lmer(DailyFlightTime~Aviary+(1|Individual),data=data)
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# This looks ok-ish
# Not sure what to do with the singular-warning

In [ ]:
# Test for differences between the groups
Model_groups<-lmer(DailyFlightTime~Aviary+(1|Individual),data=data)

cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups,ddf="Kenward-Roger")
#cat("\n------------------------- Post-hoc Tukey test --------------------------\n")
#summary(glht(Model_groups,linfct=mcp(Aviary="Tukey")))
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

### Daily distance

In [ ]:
# Load data
data<-read.csv("DATA/All_DailyDistance.csv",header=T,sep=",",na.strings=c("","NA"))

In [ ]:
# Get the total number of data points
nrow(data)

In [ ]:
# Calculate the mean and standard deviation
MeanSD<-aggregate(Distance~Aviary,data,function(x) c(mean = mean(x), sd = sd(x)))
MeanSD$Mean<-MeanSD$Distance[,1]
MeanSD$SD<-MeanSD$Distance[,2]
MeanSD$Distance<-NULL
print(MeanSD)

In [ ]:
# Check model assumptions
# Test homogeneity of variance and normality
Model_groups<-lmer(Distance~Aviary+(1|Individual),data=data)
par(mfrow=c(2,2));options(repr.plot.width = 7.5, repr.plot.height = 7.5)

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# Not sure what to do with the singular-warning
# Try log-transforming the data
data$Distance_sqrt<-sqrt(data$Distance)
data$Distance_log<-log(data$Distance)

In [ ]:
# Test homogeneity of variance and normality
Model_groups<-lmer(Distance_log~Aviary+(1|Individual),data=data)
par(mfrow=c(2,2));options(repr.plot.width = 7.5, repr.plot.height = 7.5)

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# Test homogeneity of variance and normality
Model_groups<-lmer(Distance_sqrt~Aviary+(1|Individual),data=data)
par(mfrow=c(2,2));options(repr.plot.width = 7.5, repr.plot.height = 7.5)

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# The transformations don't really help

In [ ]:
# Test for differences between the groups
Model_groups<-lmer(Distance~Aviary+(1|Individual),data=data)

cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups,ddf="Kenward-Roger")
cat("\n------------------------- Post-hoc Tukey test --------------------------\n")
summary(glht(Model_groups,linfct=mcp(Aviary="Tukey")))
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

### Daily cross-country speed

In [ ]:
# Load data
data<-read.csv("DATA/All_CrossCountrySpeedDay.csv",header=T,sep=",",na.strings=c("","NA"))

# Calculate the cross country speed in m/s
data$CCS_Straight<-1000*data$StraightDist/data$DiffTime

In [ ]:
# Get the total number of data points
nrow(data)

In [ ]:
# Calculate the mean and standard deviation
MeanSD<-aggregate(CCS_Straight~Aviary,data,function(x) c(mean = mean(x), sd = sd(x)))
MeanSD$Mean<-MeanSD$CCS_Straight[,1]
MeanSD$SD<-MeanSD$CCS_Straight[,2]
MeanSD$CCS_Straight<-NULL
print(MeanSD)

In [ ]:
# Check model assumptions
# Test homogeneity of variance and normality
Model_groups<-lmer(CCS_Straight~Aviary+(1|Individual),data=data)
par(mfrow=c(2,2));options(repr.plot.width = 7.5, repr.plot.height = 7.5)

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# Not sure what to do with the singular-warning
# Try transforming the data
data$CCS_Straight_sqrt<-sqrt(data$CCS_Straight)
data$CCS_Straight_log<-log(data$CCS_Straight)

In [ ]:
# Test homogeneity of variance and normality
Model_groups<-lmer(CCS_Straight_log~Aviary+(1|Individual),data=data)
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# Test homogeneity of variance and normality
Model_groups<-lmer(CCS_Straight_sqrt~Aviary+(1|Individual),data=data)
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# Continue with the non-transformed data

In [ ]:
# Test for differences between the groups
Model_groups<-lmer(CCS_Straight~Aviary+(1|Individual),data=data)

cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups,ddf="Kenward-Roger")
cat("\n------------------------- Post-hoc Tukey test --------------------------\n")
summary(glht(Model_groups,linfct=mcp(Aviary="Tukey")))
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

### Daily cross-country speed and wind

In [ ]:
# Load data
data<-read.csv("DATA/All_CrossCountrySpeedDay.csv",header=T,sep=",",na.strings=c("","NA"))

# Calculate the cross country speed in m/s
data$CCS_Straight<-1000*data$StraightDist/data$DiffTime

In [ ]:
# Check model assumptions
# Test homogeneity of variance and normality
Model_groups<-lmer(CCS_Straight~Aviary+WindSupport_PL+(1|Individual),data=data)
par(mfrow=c(2,2));options(repr.plot.width = 7.5, repr.plot.height = 7.5)

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# Not sure what to do with the singular-warning
# Try transforming the data
data$CCS_Straight_sqrt<-sqrt(data$CCS_Straight)
data$CCS_Straight_log<-log(data$CCS_Straight)

In [ ]:
# Test homogeneity of variance and normality
Model_groups<-lmer(CCS_Straight_log~Aviary+WindSupport_PL+(1|Individual),data=data)
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# Test homogeneity of variance and normality
Model_groups<-lmer(CCS_Straight_sqrt~Aviary+WindSupport_PL+(1|Individual),data=data)
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# Continue with the non-transformed data

In [ ]:
# Test for differences between the groups
Model_groups<-lmer(CCS_Straight~Aviary+WindSupport_PL+(1|Individual),data=data)

cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups,ddf="Kenward-Roger")
cat("\n------------------------- Post-hoc Tukey test --------------------------\n")
summary(glht(Model_groups,linfct=mcp(Aviary="Tukey")))
cat("\n--------------------------- Model selection ----------------------------\n")
Model_groups_a<-lmer(CCS_Straight~Aviary+(1|Individual),data=data)
Model_groups_w<-lmer(CCS_Straight~WindSupport_PL+(1|Individual),data=data)
Model_groups_aw<-lmer(CCS_Straight~Aviary+WindSupport_PL+(1|Individual),data=data)
anova(Model_groups_a,Model_groups_w,Model_groups_aw,ddf="Kenward-Roger")
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

### Climbing rate

In [ ]:
# Load data
data<-read.csv("DATA/All_avgClimbingB.csv",header=T,sep=",",na.strings=c("","NA"))

In [ ]:
# Get the total number of data points
nrow(data)

In [ ]:
# Check model assumptions
# Test homogeneity of variance and normality
Model_groups<-lmer(Mean_ClimbingRate~Aviary+(1|Individual),data=data)
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# Try log-transforming the data
data$ClimbingRate_sqrt<-sqrt(data$Mean_ClimbingRate)
data$ClimbingRate_log<-log(data$Mean_ClimbingRate)

In [ ]:
# Test homogeneity of variance and normality
Model_groups<-lmer(ClimbingRate_sqrt~Aviary+(1|Individual),data=data)
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# Test homogeneity of variance and normality
Model_groups<-lmer(ClimbingRate_log~Aviary+(1|Individual),data=data)
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# The sqrt transformation seems better

In [ ]:
# Test for differences between the groups
Model_groups<-lmer(ClimbingRate_sqrt~Aviary+(1|Individual),data=data)

cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups,ddf="Kenward-Roger")
cat("\n------------------------- Post-hoc Tukey test --------------------------\n")
summary(glht(Model_groups,linfct=mcp(Aviary="Tukey")))
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

### Altitude leaving thermals

In [ ]:
# Load data
data<-read.csv("DATA/All_LastAltitude.csv",header=T,sep=",",na.strings=c("","NA"))

# Get the averages per burst
data<-data.frame(data %>% group_by(tag.local.identifier,Aviary,BurstID,Burst_A,Day,Individual) %>% summarize(Altitude = mean(Last_Altitude)))

In [ ]:
# Get the total number of data points
nrow(data)

In [ ]:
# Check model assumptions
# Test homogeneity of variance and normality
Model_groups<-lmer(Altitude~Aviary+(1|Individual),data=data)
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# Try log-transforming the data
data$Altitude_log<-log(data$Altitude)

In [ ]:
# Test homogeneity of variance and normality
Model_groups<-lmer(Altitude_log~Aviary+(1|Individual),data=data)
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# That doesn't really help
# Try a quadratic transformation
data$Altitude_sqrt<-sqrt(data$Altitude)

In [ ]:
# Test homogeneity of variance and normality
Model_groups<-lmer(Altitude_sqrt~Aviary+(1|Individual),data=data)
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# That looks a bit better

In [ ]:
# Test for differences between the groups
Model_groups<-lmer(Altitude_sqrt~Aviary+(1|Individual),data=data)

cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups,ddf="Kenward-Roger")
cat("\n------------------------- Post-hoc Tukey test --------------------------\n")
summary(glht(Model_groups,linfct=mcp(Aviary="Tukey")))
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

### Gliding ground speed

In [ ]:
# Load data
data<-read.csv("DATA/All_avgGlidingB.csv",header=T,sep=",",na.strings=c("","NA"))

In [ ]:
# Get the total number of data points
nrow(data)

In [ ]:
# Check model assumptions
# Test homogeneity of variance and normality
Model_groups<-lmer(Mean_GlidingSpeed~Aviary+(1|Individual),data=data)
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# Try log-transforming the data to make the fit better
data$GlidingSpeed_log<-log(data$Mean_GlidingSpeed)

In [ ]:
# Test homogeneity of variance and normality
Model_groups<-lmer(GlidingSpeed_log~Aviary+(1|Individual),data=data)
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# This looks a lot better

In [ ]:
# Test for differences between the groups
Model_groups<-lmer(GlidingSpeed_log~Aviary+(1|Individual),data=data)

cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups,ddf="Kenward-Roger")
cat("\n------------------------- Post-hoc Tukey test --------------------------\n")
summary(glht(Model_groups,linfct=mcp(Aviary="Tukey")))
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

### Gliding airspeed

In [ ]:
# Load data
data<-read.csv("DATA/All_avgGlidingB.csv",header=T,sep=",",na.strings=c("","NA"))

In [ ]:
# Get the total number of data points
nrow(data)

In [ ]:
# Check model assumptions
# Test homogeneity of variance and normality
Model_groups<-lmer(Mean_AirSpeedPL~Aviary+(1|Individual),data=data)
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# This looks fine

In [ ]:
# Test for differences between the groups
Model_groups<-lmer(Mean_AirSpeedPL~Aviary+(1|Individual),data=data)

cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups,ddf="Kenward-Roger")
#cat("\n------------------------- Post-hoc Tukey test --------------------------\n")
#summary(glht(Model_groups,linfct=mcp(Aviary="Tukey")))
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

### Gliding wind support

In [ ]:
# Load data
data<-read.csv("DATA/All_avgGlidingB.csv",header=T,sep=",",na.strings=c("","NA"))

In [ ]:
# Get the total number of data points
nrow(data)

In [ ]:
# Check model assumptions
# Test homogeneity of variance and normality
Model_groups<-lmer(Mean_WindSupportPL~Aviary+(1|Individual),data=data)
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# Can't transform the data because of negative values

In [ ]:
# Test for differences between the groups
Model_groups<-lmer(Mean_WindSupportPL~Aviary+(1|Individual),data=data)

cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups,ddf="Kenward-Roger")
cat("\n------------------------- Post-hoc Tukey test --------------------------\n")
summary(glht(Model_groups,linfct=mcp(Aviary="Tukey")))
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

### Gliding sink

In [ ]:
# Load data
data<-read.csv("DATA/All_avgGlidingB.csv",header=T,sep=",",na.strings=c("","NA"))

In [ ]:
# Get the total number of data points
nrow(data)

In [ ]:
# Check model assumptions
# Test homogeneity of variance and normality
Model_groups<-lmer(Mean_SinkingSpeed~Aviary+(1|Individual),data=data)
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# Can't transform the data because of negative values.

In [ ]:
# Test for differences between the groups
Model_groups<-lmer(Mean_SinkingSpeed~Aviary+(1|Individual),data=data)

cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups,ddf="Kenward-Roger")
cat("\n------------------------- Post-hoc Tukey test --------------------------\n")
summary(glht(Model_groups,linfct=mcp(Aviary="Tukey")))
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

### ODBA

In [ ]:
# Load data
data<-read.csv("DATA/All_ODBA_Temp.csv",header=T,sep=",",na.strings=c("","NA"))

# Add year
data$Year<-year(data$timestamp)

# Keep only data that was collected during flight
data<-data[!(is.na(data$FlyingID)),]

In [ ]:
# Get the total number of data points
nrow(data)

In [ ]:
# Get the mean and sd
MeanSD<-aggregate(ODBA~Aviary,data,function(x) c(mean = mean(x), sd = sd(x)))
MeanSD$Mean<-MeanSD$ODBA[,1]
MeanSD$SD<-MeanSD$ODBA[,2]
MeanSD$ODBA<-NULL
print(MeanSD)

In [ ]:
# Check model assumptions
# Test homogeneity of variance and normality
Model_groups<-lmer(ODBA~Aviary+(1|Individual),data=data)
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# The residuals are not normally distributed, try log-transforming it
data$ODBA_log<-log(data$ODBA)

In [ ]:
Model_groups<-lmer(ODBA_log~Aviary+(1|Individual),data=data)
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# That looks better

In [ ]:
# Test for differences between the groups
Model_groups<-lmer(ODBA_log~Aviary+(1|Individual),data=data)

cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups,ddf="Kenward-Roger")
cat("\n------------------------- Post-hoc Tukey test --------------------------\n")
summary(glht(Model_groups,linfct=mcp(Aviary="Tukey")))
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

### Trend in ODBA over the season

In [ ]:
# Load data
data<-read.csv("DATA/All_ODBA_Temp.csv",header=T,sep=",",na.strings=c("","NA"))

# Add year
data$Year<-year(data$timestamp)

# Keep only data that was collected during flight
data<-data[!(is.na(data$FlyingID)),]

In [ ]:
# Get the total number of data points
nrow(data[data$Aviary=="CASCB",])

In [ ]:
# CASCB

In [ ]:
# Check model assumptions
# Test homogeneity of variance and normality
Model_groups<-lmer(ODBA~yday(timestamp)+(1|Individual),data=data[data$Aviary=="CASCB",])
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# The residuals are not normally distributed, try log-transforming it
data$ODBA_log<-log(data$ODBA)

In [ ]:
Model_groups<-lmer(ODBA_log~yday(timestamp)+(1|Individual),data=data[data$Aviary=="CASCB",])
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# That looks better

In [ ]:
# Test for differences between the groups
Model_groups<-lmer(ODBA_log~yday(timestamp)+(1|Individual),data=data[data$Aviary=="CASCB",])

cat("\n----------------------------- Model summary ----------------------------\n")
summary(Model_groups)
cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups,ddf="Kenward-Roger")
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

### Differences in ODBA between north and south half of segment

In [ ]:
# Load data
data<-read.csv("DATA/All_ODBA_Temp.csv",header=T,sep=",",na.strings=c("","NA"))

# Add year
data$Year<-year(data$timestamp)

# Keep only data that was collected during flight
data<-data[!(is.na(data$FlyingID)),]

# Assign the first or second half of the segment. Cutoff at 45.75 degrees north
data$Half<-NA
data[data$Lat_End>45.75,"Half"]<-1
data[data$Lat_End<=45.75,"Half"]<-2
data$Half<-as.character(data$Half)

In [ ]:
nrow(data[data$Aviary=="Affenberg",])

In [ ]:
nrow(data[data$Aviary=="CareCenter",])

In [ ]:
#-------------#
#- Affenberg -#
#-------------#

# Check model assumptions
# Test homogeneity of variance and normality
Model_groups<-lmer(ODBA~Half+(1|Individual),data=data[data$Aviary=="Affenberg",])
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# The residuals are not normally distributed, try log-transforming it
data$ODBA_log<-log(data$ODBA)

In [ ]:
# Test homogeneity of variance and normality
Model_groups<-lmer(ODBA_log~Half+(1|Individual),data=data[data$Aviary=="Affenberg",])
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# That looks a bit better

In [ ]:
# Test for differences between the first and second half
Model_groups<-lmer(ODBA_log~Half+(1|Individual),data=data[data$Aviary=="Affenberg",])

cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups,ddf="Kenward-Roger")
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

In [ ]:
#---------------#
#- Care center -#
#---------------#

# Check model assumptions
# Test homogeneity of variance and normality
Model_groups<-lmer(ODBA~Half+(1|Individual),data=data[data$Aviary=="CareCenter",])
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# Test homogeneity of variance and normality
Model_groups<-lmer(ODBA_log~Half+(1|Individual),data=data[data$Aviary=="CareCenter",])
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# That looks a bit better

In [ ]:
# Test for differences between the first and second half
Model_groups<-lmer(ODBA_log~Half+(1|Individual),data=data[data$Aviary=="CareCenter",])

cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups,ddf="Kenward-Roger")
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

In [ ]:
#---------#
#- CASCB -#
#---------#

# Check model assumptions
# Test homogeneity of variance and normality
Model_groups<-lmer(ODBA~Half+(1|Individual),data=data[data$Aviary=="CASCB",])
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# Test homogeneity of variance and normality
Model_groups<-lmer(ODBA_log~Half+(1|Individual),data=data[data$Aviary=="CASCB",])
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# That looks a bit better

In [ ]:
# Test for differences between the first and second half
Model_groups<-lmer(ODBA_log~Half+(1|Individual),data=data[data$Aviary=="CASCB",])

cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups,ddf="Kenward-Roger")
cat("\n------------------------- Post-hoc Tukey test --------------------------\n")
summary(glht(Model_groups,linfct=mcp(Half="Tukey")))
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

### ODBA during climbing vs ODBA during gliding

In [ ]:
# Load data
data<-read.csv("DATA/All_ODBA_Temp.csv",header=T,sep=",",na.strings=c("","NA"))

# Add year
data$Year<-year(data$timestamp)

# Keep only data that was collected during flight
data<-data[!(is.na(data$FlyingID)),]

# Log-transform the data
data$ODBA_log<-log(data$ODBA)

In [ ]:
# Get the number of data points
nrow(data[data$Aviary=="Affenberg",])

In [ ]:
# Get the number of data points
nrow(data[data$Aviary=="CareCenter",])

In [ ]:
# Get the number of data points
nrow(data[data$Aviary=="CASCB",])

In [ ]:
#-------------#
#- Affenberg -#
#-------------#

# Check model assumptions
# Test homogeneity of variance and normality
Model_groups<-lmer(ODBA~FlightType+(1|Individual),data=data[data$Aviary=="Affenberg",])
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# Test homogeneity of variance and normality
Model_groups<-lmer(ODBA_log~FlightType+(1|Individual),data=data[data$Aviary=="Affenberg",])
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# Test for differences between the groups
Model_groups<-lmer(ODBA_log~FlightType+(1|Individual),data=data[data$Aviary=="Affenberg",])

cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups,ddf="Kenward-Roger")
cat("\n------------------------- Post-hoc Tukey test --------------------------\n")
summary(glht(Model_groups,linfct=mcp(FlightType="Tukey")))
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

In [ ]:
#---------------#
#- Care center -#
#---------------#

# Check model assumptions
# Test homogeneity of variance and normality
Model_groups<-lmer(ODBA~FlightType+(1|Individual),data=data[data$Aviary=="CareCenter",])
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# Test homogeneity of variance and normality
Model_groups<-lmer(ODBA_log~FlightType+(1|Individual),data=data[data$Aviary=="CareCenter",])
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# Test for differences between the groups
Model_groups<-lmer(ODBA_log~FlightType+(1|Individual),data=data[data$Aviary=="CareCenter",])

cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups,ddf="Kenward-Roger")
cat("\n------------------------- Post-hoc Tukey test --------------------------\n")
summary(glht(Model_groups,linfct=mcp(FlightType="Tukey")))
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())

In [ ]:
#---------#
#- CASCB -#
#---------#

# Check model assumptions
# Test homogeneity of variance and normality
Model_groups<-lmer(ODBA~FlightType+(1|Individual),data=data[data$Aviary=="CASCB",])
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# Test homogeneity of variance and normality
Model_groups<-lmer(ODBA_log~FlightType+(1|Individual),data=data[data$Aviary=="CASCB",])
par(mfrow=c(2,2))

scatter.smooth(fitted(Model_groups),resid(Model_groups));title("Tukey-Anscombe Plot")

qqnorm(resid(Model_groups),main="normal QQ-plot,residuals");qqline(resid(Model_groups))

scatter.smooth(fitted(Model_groups),sqrt(abs(resid(Model_groups))))

qqnorm(ranef(Model_groups)$Individual[,1],main="normal QQ-plot, random effects");qqline(ranef(Model_groups)$Individual[,1])

In [ ]:
# Test for differences between the groups
Model_groups<-lmer(ODBA_log~FlightType+(1|Individual),data=data[data$Aviary=="CASCB",])

cat("\n------------------------- Anova to get p-value -------------------------\n")
anova(Model_groups,ddf="Kenward-Roger")
cat("\n------------------------- Post-hoc Tukey test --------------------------\n")
summary(glht(Model_groups,linfct=mcp(FlightType="Tukey")))
cat("\n---------------------------- Date and time -----------------------------\n")
print(Sys.time())